# Подход №2. А что, если принять как значимые признаки принять только слова из accepted_..., rejected_... ? Эта идея кажется более здравой.
# Каркас: TF-IDF матрица, причем признаки rejected_function, rejected_product умножены на -1. Затем SVD(сжатие пространства признаков)+RandomForest

In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
test=pd.read_pickle("test_df.pkl")
test.drop(columns=["keywords","html"],inplace=True)
test.head()

,id,text,accepted_function,rejected_function,accepted_product,rejected_product
0,071c2e15-5fd0-4e7d-b78d-26b7c34491e1,[We use cookies to tailor our service and ensu...,"IT support, IT managed services, IT integratio...","manufacture, production, processing, distribut...",,
1,e534df73-b385-4a12-8368-38992d3af7d3,[About us\nAbout Powernet\nPartnerships\nCusto...,"software development, database development, sy...","wholesale, distribution, retail selling, manuf...",,
2,4e38c9ff-bffe-4251-b2b2-94777c7344fe,[Work\nApproach\nPeople\nValues\nBlog\nCareers...,"general accounting, auditing, book keeping act...","manufacture, production, processing, distribut...",,
3,a7f8870c-30f8-4851-8f8b-4c16942b397f,[Menu\nMenu\nSupport\nInfrastructure\nColocati...,"Investment research and information services, ...","Construction, production and supply, Advertisi...",,
4,c062f411-ae3e-486a-9cba-2267889adb2b,[Français (French)\nEnglish (English)\n日本語 (Ja...,"general accounting, auditing, book keeping act...","manufacture, production, processing, distribut...",,


In [3]:
train=pd.read_pickle("train_df.pkl")
train.drop(columns=["keywords","html"],inplace=True)
train.head()

,id,text,accepted_function,rejected_function,accepted_product,rejected_product,target
0,b811ea46-1bc5-4a18-a64c-7504c382da44,[Release\n2020\n2019\n2018\n2017\n2020\n2019\n...,"accounting, bookeeping, auditing, finance, HR,...","manufacture production processing assembly, re...",,,2
1,b5e3f6d4-30bc-443c-a9d6-8ed41d5c33b3,[SAP Integrated Business Planning (IBP)\nSAP A...,"IT consultancy, IT support services, IT manage...","distribution, sale, trade, wholesale, retail s...",,,2
2,997ae1ab-93c4-45a9-9bd0-a49b2b952f1f,[facebook\ntwitter\nlinkedin\ninstagram\nWelco...,"general accounting, auditing, book keeping act...","manufacture, production, processing, distribut...",,,1
3,6890f2f1-bdc1-48cb-bf7f-00944eb37e77,[ANDERSON BUSINESS TRAVEL\nSpecialist Independ...,"insurance brokerage, insurance agents, travel ...","software development, wholesale, retail sale, ...",,,2
4,74a95f04-e440-4af0-850a-eec5a44ba397,[Amari Ireland Ltd\nStandards\nPeople\nQuality...,"sales agent and sales representative, product ...","manufacture, production, processing and retail...","product, motor vehicles, buses and trucks, mot...",,1


# Выборка несбалансированна:

In [4]:
for i in [0,1,2]:
    print(len(train[train['target']==i]))

1008
11557
2950


# Обратите внимание, в данных может быть гадость типа "_nan_". Ее сложно заметить, но раз уж заметили то нужно убрать.

In [5]:
features_list=['accepted_function','rejected_function','accepted_product','rejected_product']
for df in [train, test]:
    df["text"]=df["text"].apply(lambda x: x[0].lower()+x[1].lower() if (len(x)==2) else x[0].lower())
    for j in features_list:
        df[j]=df[j].apply(lambda x: x.lower())
        df[j]=df[j].apply(lambda x: "" if x=="_nan_" else x)

In [6]:
train.head()

,id,text,accepted_function,rejected_function,accepted_product,rejected_product,target
0,b811ea46-1bc5-4a18-a64c-7504c382da44,release\n2020\n2019\n2018\n2017\n2020\n2019\n2...,"accounting, bookeeping, auditing, finance, hr,...","manufacture production processing assembly, re...",,,2
1,b5e3f6d4-30bc-443c-a9d6-8ed41d5c33b3,sap integrated business planning (ibp)\nsap ad...,"it consultancy, it support services, it manage...","distribution, sale, trade, wholesale, retail s...",,,2
2,997ae1ab-93c4-45a9-9bd0-a49b2b952f1f,facebook\ntwitter\nlinkedin\ninstagram\nwelcom...,"general accounting, auditing, book keeping act...","manufacture, production, processing, distribut...",,,1
3,6890f2f1-bdc1-48cb-bf7f-00944eb37e77,anderson business travel\nspecialist independe...,"insurance brokerage, insurance agents, travel ...","software development, wholesale, retail sale, ...",,,2
4,74a95f04-e440-4af0-850a-eec5a44ba397,amari ireland ltd\nstandards\npeople\nquality\...,"sales agent and sales representative, product ...","manufacture, production, processing and retail...","product, motor vehicles, buses and trucks, mot...",,1


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(stop_words='english',analyzer="word")
texts=np.hstack([np.array(train[i]) for i in features_list])
tfidf.fit(texts)
mat_train=tfidf.transform(train['text'])
mat_test=tfidf.transform(test['text'])

In [8]:
mat_train

<15515x999 sparse matrix of type '<class 'numpy.float64'>'
	with 822212 stored elements in Compressed Sparse Row format>

In [9]:
mat_train=mat_train.toarray()
mat_test=mat_test.toarray()

In [10]:
words=list(tfidf.vocabulary_)
words

['accounting',
 'bookeeping',
 'auditing',
 'finance',
 'hr',
 'training',
 'recruitment',
 'legal',
 'support',
 'services',
 'marketing',
 'public',
 'relations',
 'management',
 'consultancy',
 'maintenance',
 'ict',
 'infrastructure',
 'managed',
 'systems',
 'integration',
 'virtualisation',
 'cloud',
 'information',
 'security',
 'database',
 'software',
 'server',
 'hosting',
 'data',
 'centre',
 'conference',
 'general',
 'book',
 'keeping',
 'activities',
 'human',
 'resource',
 'agency',
 'customization',
 'implementation',
 'installation',
 'business',
 'opinion',
 'polling',
 'market',
 'research',
 'insurance',
 'brokerage',
 'agents',
 'travel',
 'operator',
 'ticket',
 'booking',
 'administrative',
 'office',
 'commission',
 'party',
 'sales',
 'agent',
 'representative',
 'product',
 'distribution',
 'wholesale',
 'supply',
 'import',
 'export',
 'trade',
 'retail',
 'operation',
 'pharmacy',
 'medicine',
 'shops',
 'drug',
 'stores',
 'advertising',
 'strategic',
 'pla

In [11]:
for j in range(len(train)):
    temp1=train['rejected_function'].iloc[j]
    temp2=train['rejected_product'].iloc[j]
    for i in range(len(words)):
        if(temp2.find(words[i])!=-1):
            mat_train[j,i]=(-1)*mat_train[j,i]
        elif(temp1.find(words[i])!=-1):
            mat_train[j,i]=(-1)*mat_train[j,i]
for j in range(len(test)):
    temp1=test['rejected_function'].iloc[j]
    temp2=test['rejected_product'].iloc[j]
    for i in range(len(words)):
        if(temp2.find(words[i])!=-1):
            mat_test[j,i]=(-1)*mat_test[j,i]
        elif(temp1.find(words[i])!=-1):
            mat_test[j,i]=(-1)*mat_test[j,i]


In [12]:
from sklearn.decomposition import TruncatedSVD
svd=TruncatedSVD(n_components=100)
mat_train=svd.fit_transform(mat_train)
mat_test=svd.transform(mat_test)

In [13]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression,PassiveAggressiveClassifier, Perceptron, RidgeClassifier
#from sklearn.metrics import balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier

In [14]:
y_train=np.array(train["target"])

clf=RandomForestClassifier(n_estimators=1000,n_jobs=-1)
clf.fit(mat_train,y_train)
result=clf.predict(mat_test)

In [15]:
df=pd.DataFrame(data=np.hstack([np.array(test['id']).reshape(len(test),1),result.reshape(len(test),1)]),columns=['id','target'])
df.to_csv("result.csv",index=False)

# Что дает неплохой результат на Kaggle:
# public sample: 0.70946
# private sample: 0.69926